# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de grandes volúmenes de datos (Gpo 10)**

### Tecnológico de Monterrey

## Actividad 3

###  Aprendizaje supervisado y no supervisado


#### **Nombrey matrícula**

*   **A01746998** - Alexys Martín Coate Reyes


# **1. Introducción teórica**



## **Aprendizaje Supervisado**

El aprendizaje supervisado intenta predecir un output con base en una serie de datos de entrenamiento que vienen previamente clasificados de manera correcta. Este tipo de aprendizaje se asemeja al que tiene un niño pequeño que está aprendiendo a abstraer las características por medio de una guía, padre o profesor.

Entre los algoritmos supervisados más populares están los siguientes:

* Regresión Lineal
  * ***pyspark.ml.regression.LinearRegression***
* Regresión Logística
  * ***pyspark.ml.classification.LogisticRegression***
* Árboles de Decisión
  * ***pyspark.ml.classification.DecisionTreeClassifier***
  * ***pyspark.ml.regression.DecisionTreeRegressor***
* Bosques Aleatorios (Random Forests)
  * ***pyspark.ml.classification.RandomForestClassifier***
  * ***pyspark.ml.regression.RandomForestRegressor***
* Máquinas de Soporte Vectorial (SVM)
  * ***pyspark.ml.classification.LinearSVC***
* Gradient-Boosted Trees (GBTs)
  * ***pyspark.ml.classification.GBTClassifier***
  * ***pyspark.ml.regression.GBTRegressor***
* Bayes Ingenuo (Naive Bayes)
  * ***pyspark.ml.classification.NaiveBayes***


## **Aprendizaje No Supervisado**

Este tipo de modelos trabajan con un conjunto de datos no etiquetado, por lo que el mismo modelo aprende de manera automática los patrones y relaciones ocultas por si mismo.

Comunmente se utilizan en problemas de "clustering", "Reducción de dimensionalidad" o "Reglas de asociación".

Ejemplos de estos modelos son:

* K-Means
  * ***pyspark.ml.clustering.KMeans***
* Análisis de Componentes Principales (PCA)
  * ***pyspark.ml.feature.PCA***
* Factorización de Matrices No Negativas (NMF)
  * ***pyspark.ml.clustering.NNMF***
* Gaussian Mixture Models (GMM)
  * ***pyspark.ml.clustering.GaussianMixture***

# **2. Selección de los datos**

### Cargando los datos

In [98]:
# Librerias
from pyspark.sql import SparkSession

In [99]:
# Crear sesión Spark
spark = SparkSession.builder \
    .appName("EDA_Vuelos") \
    .getOrCreate()

# Leer el CSV
df = spark.read.csv("./Airline_Delay_2016-2018.csv", header=True, inferSchema=True)

# Mostrar esquema de datos
df.printSchema()

# Número total de registros
total_registros = df.count()
print(f"Número total de registros: {total_registros}")

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)

Número total de registros: 18505725


In [100]:
# Imprimiendo los 3 primeros rengloes del dataframe dataframe
df.show(3)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2016-01-01|        DL|             1248|   DTW| LAX|        1935|  1935.0|      0.0|    23.0|    1958.0|   2107.0|   13.0|        2144|  2120.0|    -24.0|      0.0|     0.0|           309.0|              285.0|   249.0|  1979.0|
|2016-01-01|        DL|             1251|   ATL| GRR|        2125|  2130.0|     

### Seleccionando variables de caracterización

In [101]:
# Variables seleccionadas
vars_particion = ["OP_CARRIER", "ORIGIN", "DEST", "CANCELLED", "DIVERTED"]

# Imrpimiendo la cantidad de valores únicos que se tiene por las columnas seleccionadas
for col in vars_particion:
    print(col, df.select(col).distinct().count())

OP_CARRIER 18
ORIGIN 362
DEST 360
CANCELLED 2
DIVERTED 2


In [ ]:
from pyspark.sql.functions import col, count, round

# Calcular frecuencia de combinaciones
combinaciones = df.groupBy(vars_particion).count()

# Total de registros
total = df.count()

# Agregar probabilidad
combinaciones = combinaciones.withColumn("probabilidad", col("count") / total)

combinaciones.show(truncate=False)

In [ ]:
from pyspark.sql.functions import when, lit

# Definir tamaño total de la muestra (por ejemplo, 1% del total)
tamaño_muestra_total = int(total * 0.01)

# Establecer un mínimo de registros por partición
minimo_por_particion = 0

# Calcular el tamaño de muestra por partición según su probabilidad
combinaciones_con_tamaño = combinaciones.withColumn(
    "tamaño_muestra",
    round(
        when(
            (col("probabilidad") * tamaño_muestra_total) < minimo_por_particion,
            lit(minimo_por_particion)
        ).otherwise(col("probabilidad") * tamaño_muestra_total)
    ).cast("int")
)

# Ordenar para visualizar mejor
combinaciones_con_tamaño = combinaciones_con_tamaño.orderBy(col("tamaño_muestra").desc())

combinaciones_con_tamaño.show(10, truncate=False)

# Calcular el tamaño final total de la muestra
tamaño_muestra_final = combinaciones_con_tamaño.agg({"tamaño_muestra": "sum"}).collect()[0][0]

print(f"Tamaño total esperado de la muestra final: {tamaño_muestra_final}")

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Unir el tamaño de muestra a cada combinación en el dataset original
df_con_muestra = df.join(
    combinaciones_con_tamaño.select(*vars_particion, "tamaño_muestra"),
    on=vars_particion,
    how="inner"
)

# Crear ventana ordenada por fecha y hora dentro de cada combinación
ventana = Window.partitionBy(*vars_particion).orderBy("FL_DATE", "CRS_DEP_TIME")

# Enumerar vuelos por combinación (ordenados por tiempo)
df_con_muestra = df_con_muestra.withColumn("row_num", row_number().over(ventana))

# Filtrar solo los primeros N registros por combinación
muestra_final = df_con_muestra.filter(col("row_num") <= col("tamaño_muestra"))

# Mostrar la muestra final
muestra_final.show(10, truncate=False)

print(f"Tamaño total de la muestra: {muestra_final.count()}")

# **3. Preparación de los datos**

### Selección de variables importantes para el modelo

Se realizará un análisis de predicción para saber que tanto un avión se retrasará. Por ello se quitarán las columnas que no aportan mucha información para el modelo.

In [ ]:
import pandas as pd

pandas_df = muestra_final.limit(50).toPandas()
display(pandas_df)

In [ ]:
# Se crea una nueva columna para verificar si
delayed_time = 10
df_muestra = muestra_final.withColumn("DELAYED", when(df.ARR_DELAY > delayed_time, 1).otherwise(0))

In [ ]:
# Variables a descartar
cols_to_drop = ["tamaño_muestra", "row_num", "OP_CARRIER_FL_NUM", "DEP_TIME","DEP_DELAY", "TAXI_OUT",
                "WHEELS_OFF", "WHEELS_ON", "TAXI_IN" , "ARR_TIME", "ARR_DELAY", "DIVERTED", "ACTUAL_ELAPSED_TIME", "AIR_TIME"]

# Variables importantes para el modelo
selected_cols = [
    "FL_DATE", "OP_CARRIER", "ORIGIN", "DEST",
    "CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME",
    "DISTANCE", "CANCELLED", "ARR_DELAY", "DELAYED"
]

# Realizando la selección de columnas importantes como datos de entrenamiento
df_raw = df_muestra.select(*selected_cols)
df_raw.show()

### Transformación de variables

In [ ]:
from pyspark.sql.functions import dayofweek, month
from pyspark.ml.feature import StringIndexer

# Extrayendo el día, mes y eliminando la columna original de fecha
df_transformed = df_raw.withColumn("FL_DAY_OF_WEEK", dayofweek("FL_DATE")) \
                     .withColumn("FL_MONTH", month("FL_DATE")) \
                     .drop("FL_DATE")

# Codificaando las variables categóricas
indexers = [
    StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_T"),
    StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_T"),
    StringIndexer(inputCol="DEST", outputCol="DEST_T")
]

for indexer in indexers:
    df_transformed = indexer.fit(df_transformed).transform(df_transformed)

# Eliminando las columnas de las variables categóricas originales
cols_to_drop = ["OP_CARRIER", "ORIGIN", "DEST"]
df_transformed = df_transformed.drop(*cols_to_drop) # Using * to unpack the list of column names

df_transformed.show()

In [ ]:
# Guardando dataset en un archivo paquet para su posterior utilización
df_transformed.write.mode("overwrite").parquet("./df_transformed.parquet")

In [ ]:
# Lectura del archivo que contiene el dataset de pyspark
df_transformed = spark.read.parquet("./df_transformed.parquet")

In [ ]:
# Imprime los valores resultantes de la limpieza y transformación de los datos
df_transformed.printSchema()

In [ ]:
# Imprime un resumen del dataframe con la limpieza de todos los datos
df_transformed.describe().toPandas()

### Limpieza de datos

In [ ]:
df_transformed.groupBy("CANCELLED").count().show()  # Cuenta cuantos vuelos cancelados hay
df_transformed.groupBy("FL_MONTH").count().show()   # Cuenta cuantos meses diferentes hay

In [ ]:
# Se elimina la columna de Cancelled y del mes ya que unicamente se cuenta con datos de un solo mes y no existen vuelos cancelasdos
drop_cols = ["CANCELLED", "FL_MONTH"]
df_transformed_2 = df_transformed.drop(*drop_cols)

In [ ]:
#Se eliminan registros con valores nulos
df_clean = df_transformed_2.dropna()

#Se eliminan columnas con valores nulos
df_clean = df_clean.na.drop()

#Se eliminan registros duplicados
df_clean = df_clean.dropDuplicates()

In [ ]:
df_clean.describe().toPandas()

### Balanceo del dataset

In [ ]:
df_clean.groupBy("DELAYED").count().show()   # Cuenta cuantos vuelos retrasados hay (Presencia de dataset desbalanceado)

Esto indica que estamos trabajando con un dataset desbalanceado, por lo que debemos utlizar métodos para balancear el dataset.

In [ ]:
# Objetivo de balanceo
target_size = 1125

# Separar las clases
df_majority = df_clean.filter(col("DELAYED") == 0)
df_minority = df_clean.filter(col("DELAYED") == 1)

# Submuestreo de clase mayoritaria (clase 0)
df_majority_sampled = df_majority.sample(False, fraction=target_size / df_majority.count(), seed=42)

# Sobremuestreo de clase minoritaria (clase 1)
ratio = int(target_size / df_minority.count())
df_minority_oversampled = df_minority
for _ in range(ratio - 1):
    df_minority_oversampled = df_minority_oversampled.union(df_minority)

# Agregar una fracción adicional si no es exacto
remaining = target_size - df_minority_oversampled.count()
if remaining > 0:
    df_minority_oversampled = df_minority_oversampled.union(
        df_minority.sample(withReplacement=True, fraction=remaining / df_minority.count(), seed=42)
    )

# Unir los datasets balanceados
df_balanced = df_majority_sampled.union(df_minority_oversampled)

# Verificación
df_balanced.groupBy("DELAYED").count().show()

In [ ]:
# Resumen final del dataset balanceado
df_balanced.describe().toPandas()

# **4. Prepraración del conjunto de entrenamiento y prueba**

### **Modelo Supervisado**

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        "OP_CARRIER_T","ORIGIN_T","DEST_T",
        "CRS_DEP_TIME","CRS_ARR_TIME","CRS_ELAPSED_TIME","DISTANCE",
        "FL_DAY_OF_WEEK",
        "ARR_DELAY","DELAYED"
    ],
    outputCol="features"
)

df_vector = assembler.transform(df_balanced)

In [ ]:
# Separación de datos de entrenamiento y prueba
spark.conf.set("spark.sql.shuffle.partitions", "200")       # Se define el valor por default del número de ejecutores
df_train, df_val_test = df_vector.randomSplit([0.7, 0.3], seed=42)
df_val, df_test = df_val_test.randomSplit([0.5, 0.5], seed=42)

# Impresion del tamaño de las particiones
print(f"""Total data: {df_vector.count()}
Training data: {df_train.count()}
Validation data: {df_val.count()}
Test data: {df_test.count()}""")

### **Modelo No Supervisado**

# **5. Construcción de modelos de aprendizaje supervisado y no supervisado**

### Modelo Supervisado - Regresión Logística

#### Entrenando el modelo

In [ ]:
from pyspark.ml.classification import GBTClassifier

# Inicializar el modelo GBTClassifier
gbt = GBTClassifier(featuresCol="features", labelCol="DELAYED", maxIter=10) # maxIter es un parámetro de ejemplo, puedes ajustarlo

# Entrenar el modelo
gbt_model = gbt.fit(df_train)

#### Evaluando el modelo

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Realizar predicciones en el conjunto de prueba
predictions = gbt_model.transform(df_test)

# Inicializar el evaluador
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="DELAYED")

# Calcular el área bajo la curva ROC (AUC)
auc = evaluator.evaluate(predictions)

print(f"Área bajo la curva ROC (AUC): {auc}")

### Modelo No Supervisado - K-Means

#### Entrenando el modelo

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Define el número de clústeres (puedes experimentar con diferentes valores)
k = 5

# Inicializa el modelo K-Means
kmeans = KMeans(featuresCol="features").setK(k).setSeed(1)

# Entrena el modelo
model = kmeans.fit(df_vector_clustering)

#### Evaluando el modelo

In [ ]:
# Realiza predicciones (asigna cada fila a un clúster)
predictions = model.transform(df_vector_clustering)

# Mostrar algunas predicciones
predictions.select("features", "prediction").show(10)

# Evaluar el modelo usando Silhouette Score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print(f"Silhouette with squared Euclidean distance = {silhouette}")

# Muestra los centros de los clústeres
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)